#Cleaning script for incoming DM form. Process to include:
# a) First clean and renaming of columns from raw form
# b) filter by watershed and ACO flight number
# c) Adding of coordinate information and importing of GNSS data 

In [1]:
#import libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
#flight/phase number?
flt_no = '1'

In [3]:
#open the raw csv, select specific columns to keep, drop additional columns
path = r"G:\ACO\2024"
data = pd.read_csv(path + r"\1_scripts\working\p1\CRU_24_P01.csv", encoding='mac-roman')

df = pd.DataFrame(data)

FileNotFoundError: [Errno 2] No such file or directory: 'CRU_24_P01.csv'

In [21]:
data = df[df["Study_Area"].notna()]
len(df.index)

400

In [22]:
df.columns

Index(['submissionid', 'submissiondatetime', 'deviceid', 'username',
       'Survey_Start_Time', 'Study_Area', 'Other_Study_Area', 'User_s_',
       'Plot_ID', 'Tube_Name', 'Pre_Survey_Notes', 'GNSS_Used_', 'GNSS_Setup',
       'Other_GNSS_Setup', 'GNSS_Status', 'GNSS_Height_Rover_to_Snow__cm_',
       'Tare_Weight__g_', 'Sampling_Design', 'Type_of_Plot',
       'Point_Observation.Cardinal_Direction',
       'Point_Observation.Distance_From_Centre__m_',
       'Point_Observation.Custom_Distance_From_Centre__m_',
       'Point_Observation.Sample_Type', 'Point_Observation.Depth__cm_',
       'Point_Observation.Depth__cm_:timestamp',
       'Point_Observation.Number_of_probe_extensions_used_',
       'Point_Observation.Depth_Final__cm_',
       'Point_Observation.Core_Length__cm_', 'Point_Observation.Plug__cm_',
       'Point_Observation.Mass___Tube__g_',
       'Point_Observation.Multi_Part_Core_',
       'Point_Observation.Multi_Part_Core_Section_Number',
       'Point_Observation.Addit

In [23]:
#put columns to keep in to a list form
columns = df[["Survey_Start_Time",
          "username", 
          "Study_Area", 
          "Other_Study_Area", 
          "User_s_", 
          "Plot_ID", 
          "Tube_Name", 
          "Pre_Survey_Notes", 
          "GNSS_Used_", 
          "GNSS_Setup", 
          "Other_GNSS_Setup",
          "GNSS_Status",
          "GNSS_Height_Rover_to_Snow__cm_", 
          "Tare_Weight__g_",
          "Type_of_Plot", 
          "Point_Observation.Cardinal_Direction", 
          "Point_Observation.Distance_From_Centre__m_",
          "Point_Observation.Custom_Distance_From_Centre__m_",
          "Point_Observation.Plot_Features", 
          "Point_Observation.Sample_Type", 
          "Point_Observation.Depth__cm_", 
          "Point_Observation.Depth__cm_:timestamp", 
          "Point_Observation.Depth_Final__cm_",
          "Point_Observation.Core_Length__cm_", 
          "Point_Observation.Plug__cm_", 
          "Point_Observation.Mass___Tube__g_",
          "Point_Observation.Multi_Part_Core_",
          "Point_Observation.Multi_Part_Core_Section_Number", 
          "Point_Observation.Additional_Measurements.Multi_Part_Core_Section_Number_copy",
          "Point_Observation.Additional_Measurements.New_Depth__cm_", 
          "Point_Observation.Additional_Measurements.New_Depth__cm_:timestamp", 
          "Point_Observation.Additional_Measurements.Core_Section_Length__cm_",
          "Point_Observation.Additional_Measurements.Plug_", 
          "Point_Observation.Additional_Measurements.Mass___Tube", 
          "Point_Observation.Core_Length_Final__cm_",
          "Point_Observation.Core_Features", 
          "Point_Observation.Depth_of_Saturation", 
          "Point_Observation.Mass_Final__g_", 
          "Point_Observation.Depth_Max",
          "Point_Observation.Retrieval____", 
          "Point_Observation.SWE__cm_", 
          "Point_Observation.Density", 
          "Point_Observation.Sample_Rating", 
          "Point_Observation.Point_Observation_Notes", 
          "Snow_Pit_Measurement.Distance_from_centre_of_plot__m_", 
          "Snow_Pit_Measurement.Depth_above_ground__cm_", 
          "Snow_Pit_Measurement.Temperature___C_",
          "Snow_Pit_Measurement.Density_Notes", 
          "Survey_End_TIme"]].copy()

In [24]:
#change the column names to manageable names, ie. 
cols = df.rename(columns={
                    "Survey_Start_Time": "plot_datetime",
                    "username": "user_name",
                    "Study_Area": "study_area",
                    "Other_Study_Area": "other_study_area",
                    "User_s_": "users",
                    "Plot_ID": "plot_id",
                    "Tube_Name": "tube_name",
                    "Pre_Survey_Notes": "pre_survey_notes",
                    "GNSS_Used_": "gnss_unit",
                    "GNSS_Setup": "gnss_setup",
                    "Other_GNSS_Setup": "other_gnss_setup",
                    "GNSS_Status": "gnss_status",
                    "GNSS_Height_Rover_to_Snow__cm_": "rover_height",
                    "Tare_Weight__g_": "tare_weight_g",
                    "Type_of_Plot": "plot_type",
                    "Point_Observation.Cardinal_Direction": "cardinal",
                    "Point_Observation.Distance_From_Centre__m_": "distance_m",
                    "Point_Observation.Custom_Distance_From_Centre__m_": "custom_distance",
                    "Point_Observation.Plot_Features": "plot_features",
                    "Point_Observation.Sample_Type": "sample_type",
                    "Point_Observation.Depth__cm_": "depth_cm",
                    "Point_Observation.Depth__cm_:timestamp": "depth_timestamp",
                    "Point_Observation.Depth_Final__cm_": "depth_final_cm",
                    "Point_Observation.Core_Length__cm_": "core_length_cm",
                    "Point_Observation.Plug__cm_": "plug_cm",
                    "Point_Observation.Mass___Tube__g_": "mass_tube_g",
                    "Point_Observation.Multi_Part_Core_": "multi_core",
                    "Point_Observation.Multi_Part_Core_Section_Number": "multi_part_core_num",
                    "Point_Observation.Additional_Measurements.Multi_Part_Core_Section_Number_copy": "multi_core_num_copy",
                    "Point_Observation.Additional_Measurements.New_Depth__cm_": "depth_cm_new",
                    "Point_Observation.Additional_Measurements.New_Depth__cm_:timestamp": "depth_cm_timestamp_new",
                    "Point_Observation.Additional_Measurements.Core_Section_Length__cm_": "core_section_length_cm",
                    "Point_Observation.Additional_Measurements.Plug_": "plug_cm_copy",
                    "Point_Observation.Additional_Measurements.Mass___Tube": "mass_tube_g_new",
                    "Point_Observation.Core_Length_Final__cm_": "core_length_final",
                    "Point_Observation.Core_Features": "core_features",
                    "Point_Observation.Depth_of_Saturation": "depth_of_saturation",
                    "Point_Observation.Mass_Final__g_": "mass_final_g",
                    "Point_Observation.Depth_Max": "depth_max",
                    "Point_Observation.Retrieval____": "retrieval",
                    "Point_Observation.SWE__cm_": "swe_cm",
                    "Point_Observation.Density": "density",
                    "Point_Observation.Sample_Rating": "sample_rating",
                    "Point_Observation.Point_Observation_Notes": "notes",
                    "Snow_Pit_Measurement.Distance_from_centre_of_plot__m_": "snow_pit_distance_from_centre",
                    "Snow_Pit_Measurement.Depth_above_ground__cm_": "snow_pit_depth_above_ground",
                    "Snow_Pit_Measurement.Temperature___C_": "snow_pit_temperature",
                    "Snow_Pit_Measurement.Density_Notes": "snow_pit_density",
                    "Survey_End_TIme": "survey_end"
                    }, inplace=True)

In [25]:
new_df = df.assign(aco_flight_number= str(flt_no))

In [26]:
aco_pos = new_df.pop('aco_flight_number')
new_df.insert(0, 'aco_flight_number', aco_pos)

In [27]:
new_df

,aco_flight_number,submissionid,submissiondatetime,deviceid,user_name,plot_datetime,study_area,other_study_area,users,plot_id,...,swe_cm,density,sample_rating,notes,Point_Observation.Picture,snow_pit_distance_from_centre,snow_pit_depth_above_ground,snow_pit_temperature,snow_pit_density,survey_end
0,1,91645433,2024-05-10 19:45:44 -07:00,iPad_B21591CC-BEAE-44CA-992E-DDB5854CED2F,CHRL-Nikolai Schirmer,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
1,1,91645433,2024-05-10 19:45:44 -07:00,iPad_B21591CC-BEAE-44CA-992E-DDB5854CED2F,CHRL-Nikolai Schirmer,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,...,52.575,0.431,Good,NaN,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
2,1,91645433,2024-05-10 19:45:44 -07:00,iPad_B21591CC-BEAE-44CA-992E-DDB5854CED2F,CHRL-Nikolai Schirmer,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
3,1,91645433,2024-05-10 19:45:44 -07:00,iPad_B21591CC-BEAE-44CA-992E-DDB5854CED2F,CHRL-Nikolai Schirmer,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
4,1,91645433,2024-05-10 19:45:44 -07:00,iPad_B21591CC-BEAE-44CA-992E-DDB5854CED2F,CHRL-Nikolai Schirmer,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,...,NaN,NaN,Good,NaN,https://drive.google.com/uc?id=1JwCoU5g-fM9hbK...,NaN,NaN,NaN,NaN,5/10/2024 10:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,91645476,2024-05-10 19:50:04 -07:00,iPad_F38B7DE9-9E96-46F1-8A21-B55333832D55,CHRL-Warren Miller,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,1,91645476,2024-05-10 19:50:04 -07:00,iPad_F38B7DE9-9E96-46F1-8A21-B55333832D55,CHRL-Warren Miller,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,1,91645476,2024-05-10 19:50:04 -07:00,iPad_F38B7DE9-9E96-46F1-8A21-B55333832D55,CHRL-Warren Miller,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,1,91645476,2024-05-10 19:50:04 -07:00,iPad_F38B7DE9-9E96-46F1-8A21-B55333832D55,CHRL-Warren Miller,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,...,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df = new_df.drop(columns=['submissionid', 
                                'submissiondatetime', 
                                'deviceid', 
                                'user_name', 
                                'Sampling_Design',
                                'Point_Observation.Abbreviations', 
                                'Point_Observation.Picture', 
                                'Point_Observation.Number_of_probe_extensions_used_'])

#Take the cleaned form and filter out by watershed
#export summary statistics

In [29]:
#join depth_final_cm and depth_max - name snow_depth
columns = df.loc[:, ['depth_final_cm', 'depth_max']]
columns.fillna(0, inplace=True)
df= df.assign(snow_depth = (columns['depth_final_cm']) + (columns['depth_max']))

In [22]:
#titles = list(df.columns)
#titles

In [30]:
df.insert(25, "snow_depth", df.pop("snow_depth"))

In [31]:
len(df.index)

400

In [32]:
#export the clean raw device magic form to a csv
df.to_csv("processing_data\DM_trip" + str(flt_no) + "_clean_cru.csv", index = False)

In [33]:
#filter by watershed: "Russell Creek", "Cruickshank",or "Englishman"
cru_df =df[df['study_area'] == 'Cruickshank']
eng_df = df[df['study_area'] == 'Englishman']
rus_df = df[df['study_area'] == 'Russell Creek']
mv_df = df[df['study_area'] == 'Metro Vancouver']

In [34]:
cru_df

,aco_flight_number,plot_datetime,study_area,other_study_area,users,plot_id,tube_name,pre_survey_notes,gnss_unit,gnss_setup,...,retrieval,swe_cm,density,sample_rating,notes,snow_pit_distance_from_centre,snow_pit_depth_above_ground,snow_pit_temperature,snow_pit_density,survey_end
0,1,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,CHRL 1,Big disturbance on south due to soft snow. 20-...,NaN,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
1,1,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,CHRL 1,Big disturbance on south due to soft snow. 20-...,NaN,NaN,...,93.0,52.575,0.431,Good,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
2,1,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,CHRL 1,Big disturbance on south due to soft snow. 20-...,NaN,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
3,1,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,CHRL 1,Big disturbance on south due to soft snow. 20-...,NaN,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
4,1,5/10/2024 10:11,Cruickshank,NaN,Ak bf tm,S2F,CHRL 1,Big disturbance on south due to soft snow. 20-...,NaN,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,5/10/2024 10:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,CHRL 1,New pokes,No already surveyed,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN
396,1,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,CHRL 1,New pokes,No already surveyed,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN
397,1,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,CHRL 1,New pokes,No already surveyed,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN
398,1,5/10/2024 14:04,Cruickshank,NaN,"Rw, lm, sm",S5O,CHRL 1,New pokes,No already surveyed,NaN,...,NaN,NaN,NaN,Good,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#group by "plot_id", export csv of summary stats for snow. 
#Stats that are nice to include are: 
#snow depth:count, average, st dev; density: count, avg, st dev, & SWE : count, avg, st.dev



###CRUICKSHANK###

cru_sum = cru_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Cruickshank\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
cru_sum.to_csv(path + "\Cruickshank\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_cru_sum.csv", index = True)

In [23]:
###ENGLISHMAN###

eng_sum = eng_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Englishman\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
eng_sum.to_csv(path + "\Englishman\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_eng_sum.csv", index = True)

In [17]:
###TSITIKA####


rus_sum = rus_df.groupby(['aco_flight_number',
    'plot_id']).agg({
    "snow_depth": ["mean", "median", "std", "count"],
    "density": ["mean", "median", "std", "count"],
    "swe_cm": ["mean", "median", "std", "count"]
})

#check if directory exists
if not os.path.isdir(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no)):
    #if the folder path is not presnet, then create it
    os.makedirs(path + r"\Tsitika\4_field_data\plots\working\P" + str(flt_no))
    
#Export the filtered summary stats to created folder
rus_sum.to_csv(path + "\Tsitika\\4_field_data\plots\working\P" + str(flt_no) + "\DM_trip" + str(flt_no) +"_rus_sum.csv", index = True)